# Programação Linear

## Programação Linear

Na Programação Linear, o termo "programação" não está ligado diretamente apenas com o desenvolvimento de códigos, mas sim, com a programação em geral, como um algoritmo que mostra uma sequência de passos e instruções que resolve um problema. A Programação Linear é um modelo matemático criado para a melhor tomada de decisão afim de encontrar uma solução otimizada para o problema.

Um modelo de programação envolve um problema inicial como o de minimizar uma função:  $3x_1 + 2x_2$, com essa função, precisamos analisar os parâmetros definidos, por exemplo:

1.  $x_1$ e $x_2 \geq 0$
2.  $x_1 + x_2 \geq 4$
3.  $x_1 \leq 3$
4.  $x_2 \leq 2$

A partir das informações passadas, podemos tomar uma decisão para a função.

#Modelagem de problemas de otimização e implementação em AMPL





In [ ]:
#ao executar este código, instalamos o suporte ao AMPL no notebook
!pip install -q amplpy
from amplpy import tools, AMPL
Ampl = tools.ampl_notebook(
    modules=["highs"],      # pick from over 20 modules including most commercial and open-source solvers
    license_uuid="default") # your license UUID

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://ampl.com/colab).


# Problema da Dieta

> Os dados utilizados aqui foram retirados de [[Tirone, 2019]](https://imef.furg.br/images/stories/Monografias/Matematica_aplicada/2019/2019-2_Giulia.pdf)

Sua equipe está atuando no setor de planejamento do refeitório de uma fábrica, conjuntamente com a nutricionista responsável pelo estabelecimento. Diariamente, a empresa serve milhares de refeições aos trabalhadores, de tal modo que uma grande quantia é gasta todos os meses com este serviço.

No entando, há a necessidade de se **reduzir os custos** no preparo da refeições **sem prejudicar a saúde** dos trabalhadores. Logo, optou-se por criar um modelo de _programação linear_ para auxiliar no processo de escolha dos alimentos que serão oferecidos aos trabalhadores em um dia específico, bem como sua respectiva quantidade.

A nutricionista estabeleceu quantidades mínimias para certos nutrientes, vitaminas e minerais que um adulto deve ingerir ao longo de um dia:

* Proteínas: 46g ou mais;
* Cálcio: 1g ou mais;
* Sódio: 1.5g ou mais;
* Ferro: 0.018g ou mais;
* Vitaminas: 0.771g ou mais;

A quantidade máxima de calorias ingeridas também foi fixada em 2000kcal.

Os alimentos disponíveis no estoque e suas respectivas informações são dados <a href="https://docs.google.com/spreadsheets/d/1SNGjaFgBLFy-NqaUT2pnF84GIN4QShZaXuHBF_JTX9M/edit?usp=sharing">aqui</a>.

## Tabelas separadas
* [Tabela de nutrientes](https://docs.google.com/spreadsheets/d/e/2PACX-1vTRwIDujyFlL_zdMailPitroLTHkMKO5bhUGtMT-VKWLVZLJ35YYOxcreCCOEuJbdGeCdnwZTkZM5WW/pubhtml?gid=0&single=true);
* [Tabela de custos](https://docs.google.com/spreadsheets/d/e/2PACX-1vTRwIDujyFlL_zdMailPitroLTHkMKO5bhUGtMT-VKWLVZLJ35YYOxcreCCOEuJbdGeCdnwZTkZM5WW/pubhtml?gid=636065252&single=true);
* [Tabela de calorias](https://docs.google.com/spreadsheets/d/e/2PACX-1vTRwIDujyFlL_zdMailPitroLTHkMKO5bhUGtMT-VKWLVZLJ35YYOxcreCCOEuJbdGeCdnwZTkZM5WW/pubhtml?gid=1079156719&single=true);
* [Tabela de limites](https://docs.google.com/spreadsheets/d/e/2PACX-1vTRwIDujyFlL_zdMailPitroLTHkMKO5bhUGtMT-VKWLVZLJ35YYOxcreCCOEuJbdGeCdnwZTkZM5WW/pub?gid=1651309023&single=true&output=csv).

## Questionamentos do problema:

* Quais são os parâmetros?
* Quais são as variáveis de decisão e qual seu tipo?
* Qual a função objetivo?
* Quais as restrições?

# Formulação Dieta
---
## 1. Formulação expandida
**Objetivo**

$minimize$ 0,93$x_1$ + 5,24$x_2$ + … + 1,6$x_{13}$ $→ Custo$

**Restrições**

136$x_1$ + 85$x_2$ + … + 99$x_{13} \leq 2000 kcal$ $→ Calorias$

4,6$x_1$ + 17,2$x_2$ + … + 3,9$x_{13} \geq 46g$ $→ Proteínas$

0,6$x_1$ + 0,3$x_2$ + … + 0,14$x_{13} \geq 1g$ $→ Cálcio$

0,276$x_1$ + 0,013$x_2$	+ … +	0,053$x_{13} \geq 1,5g$ $→ Sódio$

0,100$x_1$ + 0,010$x_2$	+ … +	0,000$x_{13} \geq 0,018g$ $→ Ferro$

0$x_1$ + 0,01$x_2$ + … + 0,02$x_{13} \geq 0,771g$ $→ Vitaminas$


\\

## 2. Formulação Compacta
O problema da escolha da dieta consiste em escolher quais alimentos de um conjunto de $n$ items devem formar a dieta do refeitório de uma fábrica. Para isso, é preciso respeitar o limite de ingestão calórica de ${2000kcal}$

Parâmetros:
- $m$: Total de alimentos = 13;
- $n$: Total de nutrientes = 5;
- $nutriente_{ij}$: cada um dos elementos da matriz de nutrientes;
- $custo_i$: custo de cada elemento $i$;
- $caloria_i$: é a quantidade de cada caloria de cada alimento;
- $l_i$: é a quantidade mínima de cada nutriente;
- $u_i$: = é a quantidade máxima de cada nutriente;
- $max_calorias$: é a quantidade máxima de calorias permitida;
- $min_calorias$: é a quantidade mínima de calorias permitida;
- $nome_alimento$: nome de cada alimento.

Variáveis:
- $x$: variável que indica a quantidade de cada alimento.

Objetivo:

$$
min~CUSTO: \sum_{i=1}^{m}custo_i \times x_i
$$

Restrições:

$$
r_1: \text{min_calorias} \leq \sum_{i=1}^{m}calorias_i\times x_i \leq max_calorias, \forall j \in n
$$

$$
r_2: l_j \leq \sum_{i=1}^{m}nutrientes_{ij}\times x_i \leq u_j, \forall
$$


## Crie seu arquivo .dat

In [ ]:
%%writefile dieta.dat

param m := 13;
param n:= 5;
param max_calorias := 2000;
param min_calorias := 0;

param : nome_nutriente   l       u :=
    1   "proteínas (g)"  46      Infinity
    2   "cálcio (g)"     1       Infinity
    3   "sódio (g)"      1.5     Infinity
    4   "ferro (g)"      0.018   Infinity
    5   "vitaminas (g)"  0.77    Infinity;


param : nome_ingrediente   custo :=
     1  "pão integral"     0.93
     2  "queijo cottage"   5.24
     3  "mamão"            1.29
     4  "nozes"            3.05
     5  "salada crua"      0.5
     6  "feijão"           0.63
     7  "arroz integral"   0.09
     8  "frango grelhado"  0.89
     9  "maçã"             1.95
     10 "tapioca"          0.21
     11 "ovo"              0.99
     12 "atum ralado"      0.61
     13 "iorgute"          1.69;

param : nome_ingrediente     calorias :=
     1  "pão integral"       136
     2  "queijo cottage"     85
     3  "mamão"              45
     4  "nozes"              100
     5  "salada crua"        25
     6  "feijão"             132
     7  "arroz integral"     25
     8  "frango grelhado"    165
     9  "maçã"               72
     10 "tapioca"            68
     11 "ovo"                77
     12 "atum ralado"        17
     13 "iorgute"            99;


param nutriente: 1     2    3       4      5 :=
     1           4.6  0.06  0.276   0.1    0
     2           17.2 0.03  0.013   0.01   0.01
     3           0.8  0.02  0.0055  0.01   0.77
     4           2.3  0     0       0      0
     5           1.5  0.02  0.08    0.03   0.93
     6           8.8  0.03  0.001   0.12   0
     7           0.5  0     0       0      0
     8           31   0.02  0.074   0.06   0
     9           0.3  0.01  0.001   0.01   0.12
     10          0    0     0.037   0.02   0
     11          6.2  0.03  0.139   0.03   0.06
     12          3    0     0.069   0      0
     13          3.9  0.14  0.053   0      0.02;

Overwriting dieta.dat


## Crie seu arquivo .mod

In [ ]:
%%writefile dieta.mod
param m > 0;
param n > 0;
param min_calorias >= 0;
param max_calorias >= 0;
param calorias{1..m} >= 0;
param nome_ingrediente{1..m} symbolic;
param nome_nutriente{1..n} symbolic;
param nutriente{1..m, 1..n} >= 0;
param custo{1..m} >= 0;
param l{1..n} >= 0;
param u{1..n} >= 0;

var x{1..m} >= 0;

minimize Z: sum{i in 1..m}custo[i]*x[i];

subject to r1{j in 1..n}: l[j] <= sum{i in 1..m}nutriente[i, j]*x[i] <= u[j];

subject to r2{j in 1..n}: min_calorias <= sum{i in 1..m}calorias[i]*x[i] <=max_calorias;

Overwriting dieta.mod


## Crie seu arquivo .run ou execute o ampl_eval

In [ ]:
# %%ampl_eval

%%writefile dieta.run

reset;

model "dieta.mod";
data "dieta.dat";
option solver highs;

solve;

for {i in 1..m}
{
    if x[i] > 0 then
      printf "%s %f\n", nome_ingrediente[i], x[i];
};

display sum{i in 1..m}calorias[i]*x[i];

Overwriting dieta.run


In [ ]:
!ampl dieta.run

HiGHS 1.8.1: HiGHS 1.8.1: optimal solution; objective 13.22738288
4 simplex iterations
0 barrier iterations
pão integral 4.177816
salada crua 0.715579
frango grelhado 0.171900
iorgute 5.225582
sum{i in 1 .. m} calorias[i]*x[i] = 1131.77



# Python

## Aquisição de dados

In [ ]:
import pandas as pd
import numpy as np

links = {
    "nutrientes": "https://www.google.com/url?q=https%3A%2F%2Fdocs.google.com%2Fspreadsheets%2Fd%2Fe%2F2PACX-1vTRwIDujyFlL_zdMailPitroLTHkMKO5bhUGtMT-VKWLVZLJ35YYOxcreCCOEuJbdGeCdnwZTkZM5WW%2Fpub%3Fgid%3D0%26single%3Dtrue%26output%3Dcsv",
    "calorias": "https://www.google.com/url?q=https%3A%2F%2Fdocs.google.com%2Fspreadsheets%2Fd%2Fe%2F2PACX-1vTRwIDujyFlL_zdMailPitroLTHkMKO5bhUGtMT-VKWLVZLJ35YYOxcreCCOEuJbdGeCdnwZTkZM5WW%2Fpub%3Fgid%3D1079156719%26single%3Dtrue%26output%3Dcsv",
    "custos": "https://www.google.com/url?q=https%3A%2F%2Fdocs.google.com%2Fspreadsheets%2Fd%2Fe%2F2PACX-1vTRwIDujyFlL_zdMailPitroLTHkMKO5bhUGtMT-VKWLVZLJ35YYOxcreCCOEuJbdGeCdnwZTkZM5WW%2Fpub%3Fgid%3D636065252%26single%3Dtrue%26output%3Dcsv",
    "limites": "https://www.google.com/url?q=https%3A%2F%2Fdocs.google.com%2Fspreadsheets%2Fd%2Fe%2F2PACX-1vTRwIDujyFlL_zdMailPitroLTHkMKO5bhUGtMT-VKWLVZLJ35YYOxcreCCOEuJbdGeCdnwZTkZM5WW%2Fpub%3Fgid%3D1651309023%26single%3Dtrue%26output%3Dcsv"
}

db_nutrientes = pd.read_csv(links["nutrientes"], sep=",", index_col='Unnamed: 0')
db_calorias = pd.read_csv(links["calorias"], sep=",", index_col='Unnamed: 0')
db_custos = pd.read_csv(links["custos"], sep=",", index_col='Unnamed: 0')
db_limites = pd.read_csv(links["limites"], sep=",", index_col='Unnamed: 0')

#db_nutrientes
db_calorias
#db_custos
#db_limites

,calorias (kcal)
pão integral,136.0
queijo cottage,85.0
mamão,45.0
nozes,100.0
salada crua,25.0
feijão,132.0
arroz integral,25.0
frango grelhado,165.0
maçã,72.0
tapioca,68.0


## Utilização `amplpy`

In [ ]:
ampl = AMPL()
ampl.reset()

# Definir parâmetros
m = db_nutrientes.shape[0]
n = db_nutrientes.shape[1]
l = db_limites['limite inferior'].values
u = db_limites['limite superior'].values
max_calorias = 2000
min_calorias = 0

nome_ingrediente = db_nutrientes.index.values
nome_nutriente = db_nutrientes.columns.values
calorias = db_calorias.values
custo = db_custos.values
nutriente = db_nutrientes.values

# Carregar modelo
ampl.read('dieta.mod')

# Passar parâmetros
ampl.param['m'] = m
ampl.param['n'] = n
ampl.param['l'] = l
ampl.param['u'] = u
ampl.param['max_calorias'] = max_calorias
ampl.param['min_calorias'] = min_calorias
ampl.param['nome_ingrediente'] = nome_ingrediente
ampl.param['nome_nutriente'] = nome_nutriente
ampl.param['calorias'] = calorias
ampl.param['custo'] = custo
ampl.param['nutriente'] = nutriente

# Configurar solver e opções AMPL
ampl.setOption('solver', 'highs')
ampl.setOption('highs_options', "timelim=620 debug=1 miploglev=2 outlev=1")

# Resolver o problema
ampl.solve()

HiGHS 1.8.1:   lim:time = 620
  tech:debug = 1
  tech:miploglev = 2
  tech:outlev = 1
Running HiGHS 1.8.1 (git hash: 4a7f24a): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e-03, 2e+02]
  Cost   [9e-02, 5e+00]
  Bound  [0e+00, 0e+00]
  RHS    [2e-02, 2e+03]
Presolving model
10 rows, 13 cols, 112 nonzeros  0s
6 rows, 13 cols, 60 nonzeros  0s
6 rows, 13 cols, 60 nonzeros  0s
Presolve : Reductions: rows 6(-4); columns 13(-0); elements 60(-52)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 5(248.396) 0s
          4     1.3227382884e+01 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 4
Objective value     :  1.3227382884e+01
Relative P-D gap    :  4.0288170115e-16
HiGHS run time      :          0.00
HiGHS 1.8.1: optimal solution; objective 13.22738288
4 simplex iterations
0 

## Exibição de resultados

In [ ]:
# Imprimir os resultados
resultado = {}

for i in range(1, m + 1):
    resultado[nome_ingrediente[i -1]] = ampl.var['x'][i].value()

resultado = pd.Series(resultado)
resultado.index.name = 'INGREDIENTE'
resultado.name = 'QUANTIDADE'
display(resultado[resultado > 0].round(3))

ampl.display('sum{i in 1..m}calorias[i]*x[i]')
ampl.display('Z')

,QUANTIDADE
INGREDIENTE,
pão integral,4.178
salada crua,0.716
frango grelhado,0.172
iogurte,5.226


sum{i in 1 .. m} calorias[i]*x[i] = 1131.77

Z = 13.2274



## Reflexões

1. Quais as limitações da solução apresentada para o problema?
2. O que poderia ser melhorado?
3. Em um cenário um pouco mais realista, como poderia ser pensado o cardápio de um refeitório como este descrito?
4. Este modelo poderia ser aplicado na sua casa ou trabalho?